#### Base Approach  ~0.94
1. Join all columns that have text
1. Clean html tags
1. Replace non english letters with equivalent Å -> A
1. lowercase
1. Google BERT Base model with 256 tokens and fine tuned 2 layers + Dense 256 layer on top

#### More thorrow cleunup + BERT Custom dictionary ~0.98
1. Remove weight references like 600g
1. Remove size references (150cm x 16cm) but keep words like length, diameter etc.
1. Fix incorrect labels in train dataset (I only found 1, but there are more)
1. Read "Start & Run an Adult Boutique" book and write all adult toys related words into bert Dictionary file 
https://books.google.ca/books?id=LMQhCgAAQBAJ&pg=PT157&lpg=PT157&dq=adult+toys+vocabulary&source=bl&ots=OneH8Cmfa2&sig=ACfU3U3_8mllbStNXxpfJ6WLhrTK3IdiFQ&hl=en&sa=X&redir_esc=y#v=onepage&q=adult%20toys%20vocabulary&f=false


####  ~0.99+

1. Add some common intentional misspellings like "didlo"
1. Add training samples based on those words with label 1 to force BERT pay higher attention
1. Count Å -> A replacements and add special custom word "nel" if there are too many of those. Usually this means people are trying to cheat and hide real word. (not sure if it helps)
1. Train 3 top BERT layers instead of 2 (although usually does not converge, some luck required)
1. Increase number of token from 256 to 512 to catch more words

In [36]:
import import_ipynb
import tensorflow as tf
import pandas as pd
import tensorflow_hub as hub
import os
import re
from sklearn.metrics import confusion_matrix, accuracy_score
import numpy as np
from bert.tokenization import FullTokenizer
from tqdm import tqdm_notebook
from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split
import bertkeras as bk
import BERTDictionary as bd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import pyplot
from unidecode import unidecode
from scipy.stats import norm, boxcox
pd.options.display.max_colwidth = 1000
sns.set()

In [37]:
df= pd.read_csv('../dataset/training.csv', index_col='ID')
labels = df['label']
drop_columns = [ '﻿ASIN','parent_asin','Product Group Description', 
                      'product_name', 'brand_code', 'brand_name']
df = df.drop(columns=drop_columns)
df = df.fillna("")

cols = df.columns.tolist()
cols.remove('label')
text = df[cols].apply(' '.join, axis=1)


In [38]:
html_re = re.compile('<[^>]*>')
garbage_re = re.compile('[^0-9a-z,\'\.]+') 
weight_re = re.compile('[\d.]+[ ]?(kg+|lbs+|g+|lb+)')


# check if it actually helps, tests https://regex101.com/r/xwnV5l/1
size_re = re.compile('[\d.]+([ cminch]+)?[ x*]+[\d.]+([ cminch]+)?([ x*]+[\d.]+)?([ cminch]+)')

# play around with garbage_threshhold 
garbage_threshhold = 0.003
def html_cleanup(html):
    return re.sub(html_re, ' ', html)
def lower_case(txt):
    return txt.lower()
def split_by_garbage(txt):
    return re.sub(garbage_re, ' ', txt)
def decode_chars(txt):
    garbage_fiteted = re.sub(garbage_re, '', txt)
    decoded_txt = unidecode(txt)
    return decoded_txt if len(garbage_fiteted)/len(txt) < garbage_threshhold \
        else '[NEL] ' + decoded_txt 

def cleanup_size(txt):
    return re.sub(size_re, ' ', txt).replace('size:', '')

def cleanup_weight(txt):
    return re.sub(weight_re, ' ', txt)
    
def cleanup(text):
    res = text.apply(html_cleanup).apply(decode_chars).apply(lower_case) \
              .apply(split_by_garbage).apply(cleanup_size) \
              .apply(cleanup_weight)
    return res

In [39]:
texts = cleanup(text)

In [40]:
df = pd.DataFrame( {"texts" :texts, "labels": labels} )

In [41]:
# there is error in training dataset
df.loc[196,'labels'] =1


In [42]:
txt_train, txt_test, lbl_train, lbl_test = train_test_split(
    texts, labels, test_size=0.0002, random_state=42)

In [43]:
k=10000
arr = np.array_split(bd.adult_toys_vocab, 25)
for a in arr :
    print(a)
    w = ' '.join(a)
    txt_train = txt_train.append(pd.Series([w], index=[k]))
    lbl_train = lbl_train.append(pd.Series([1], index=[k]))
    k = k+1


['[NEL]' 'nel' 'porn' 'nubby']
['lesbian' 'gay' 'transvestite' 'transgender']
['lube' 'lubricant' 'sex' 'intercourse']
['fuck' 'vibrate' 'suction' 'lick']
['suck' 'gag' 'dildo' 'vibrator']
['condom' 'anal' 'plug' 'snake']
['trembler' 'pulsator' 'lockaid' 'strap']
['buzzer' 'rabbit' 'egg' 'penetrator']
['dual' 'kegel' 'ben']
['wa' 'stimulator' 'masturbator']
['lasso' 'bondage' 'cuff']
['pasties' 'strapon' 'didlo']
['dido' 'dilo' 'dlo']
['device' 'gadget' 'pleasure']
['intimate' 'last' 'aroused']
['climax' 'enjoyment' 'orgasm']
['bdsm' 'sadism' 'submission']
['masochism' 'realistic' 'stimulate']
['squirt' 'virginity' 'dick']
['cock' 'penis' 'spot']
['butt' 'ass' 'nipple']
['clitor' 'pelvic' 'vagina']
['pussy' 'anus' 'prostate']
['rectum' 'ventral' 'scrotum']
['tongue' 'perineum' 'boob']


In [44]:
tokenizer = bk.create_tokenizer_from_hub_module(vocab_file='vocab.txt')
max_seq_length = 512

using vocab file from vocab.txt


In [45]:
# Convert data to InputExample format
train_examples = bk.convert_text_to_examples(txt_train, lbl_train.values)
test_examples = bk.convert_text_to_examples(txt_test, lbl_test.values)

In [46]:

# Convert to features
(train_input_ids, train_input_masks, train_segment_ids, train_labels 
) = bk.convert_examples_to_features(tokenizer, train_examples, max_seq_length=max_seq_length)
(test_input_ids, test_input_masks, test_segment_ids, test_labels
) = bk.convert_examples_to_features(tokenizer, test_examples, max_seq_length=max_seq_length)

In [47]:
train_examples[0].__dict__

{'guid': None,
 'text_a': " nel lqqbstorage celtic decor,grommet window curtain,head of legend dragon with ethnic african ornate effects on grunge backdrop myth celtic design, noise reducing curtain,multi multifunction 1.curtains is delicately designed, allowing you to decorate your windows with great styles,protect your furniture, floors and walls from the sun. 2.featuring vibrant colored pattern, this eye soothing curtain can match interiors of any room. 3.soft and durable window curtains has a very soft hand feel and drapery, high quality material, durable 10 years. 4.privacy and relax your family and friends can enjoy movie nights or sports game without worrying about outside light and noise. 5.energy efficient innovative triple weave construction insulate your sunny or frozen window, give your air conditioner and energy bill a break. 6.reduce noise densely woven fabric acts as an additional sound barrier,it reduces outside noises and create a quiet and peaceful environment. widely

In [48]:
def build_model(max_seq_length): 
    in_id = tf.keras.layers.Input(shape=(max_seq_length,), name="input_ids")
    in_mask = tf.keras.layers.Input(shape=(max_seq_length,), name="input_masks")
    in_segment = tf.keras.layers.Input(shape=(max_seq_length,), name="segment_ids")
    bert_inputs = [in_id, in_mask, in_segment]
    def auc(y_true, y_pred):
        auc = tf.metrics.auc(y_true, y_pred)[1]
#     K.get_session().run(tf.local_variables_initializer())
        return auc
    bert_output = bk.BertLayer(n_fine_tune_layers=2, pooling="first")(bert_inputs)
    dense = tf.keras.layers.Dense(256, activation='relu')(bert_output)
    pred = tf.keras.layers.Dense(1, activation='sigmoid')(dense)
    
    model = tf.keras.models.Model(inputs=bert_inputs, outputs=pred)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[auc])
    model.summary()
    
    return model

In [49]:
METRIC_THRESHOLD = 0.99
LOSS_THRESHOLD = 0.015
class myCallback(tf.keras.callbacks.Callback): 
    def on_batch_end(self, batch, logs): 
        if(logs.get('auc') > METRIC_THRESHOLD and logs.get('loss')<LOSS_THRESHOLD):   
          print("\nReached %2.2f%% auc, so stopping training!!" %(METRIC_THRESHOLD*100)) 
          self.model.stop_training = True

In [50]:
model = build_model(max_seq_length)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          (None, 512)          0                                            
__________________________________________________________________________________________________
input_masks (InputLayer)        (None, 512)          0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        (None, 512)          0                                            
__________________________________________________________________________________________________
bert_layer_2 (BertLayer)        (None, 768)          110104890   input_ids[0][0]                  
                                                                 input_masks[0][0]                
          

In [51]:
# Initialize session
batch_size = 24
sess = tf.Session()
# Instantiate variables
bk.initialize_vars(sess)
tb = tf.compat.v1.keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=batch_size, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None, embeddings_data=None, update_freq='batch')
stop98 = myCallback()
model.fit(
    [train_input_ids, train_input_masks, train_segment_ids], 
    train_labels,
    validation_data=([test_input_ids, test_input_masks, test_segment_ids], test_labels),
    callbacks = [ tb, stop98], #, 
    epochs=8,
    batch_size=batch_size
)

Train on 5509 samples, validate on 2 samples
Epoch 1/8


ResourceExhaustedError: OOM when allocating tensor with shape[12288,768] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node bert_layer_2/bert_layer_2_module_apply_tokens/bert/encoder/layer_10/output/LayerNorm/moments/SquaredDifference}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[{{node metrics_2/auc/div_no_nan}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


In [ ]:
model.save('restricted.h5')

In [ ]:
df_subm= pd.read_csv('../dataset/public_test_features.csv', index_col='ID')
df_subm = df_subm.drop(columns=drop_columns)
df_subm = df_subm.fillna("")

text_subm = df_subm[cols].apply(' '.join, axis=1)


In [ ]:
texts_subm = cleanup(text_subm)

In [ ]:
subm_examples = bk.convert_text_to_examples(texts_subm, [0]*len(texts_subm))

In [ ]:
(subm_input_ids, subm_input_masks, subm_segment_ids, subm_labels
) = bk.convert_examples_to_features(tokenizer, subm_examples, max_seq_length=max_seq_length)

In [ ]:
subm_examples[0].text_a

In [ ]:
testCount = len(subm_input_ids)
def pickMax(arr):
  m = max(arr)
  return [x for x in arr]

threshold = 0.4
def tolabel(arr):
  # return int(max(arr)>=threshold)
  return max(arr)
    


In [ ]:
predictions = model.predict([subm_input_ids[0:testCount], 
                                subm_input_masks[0:testCount], 
                                subm_segment_ids[0:testCount]])

In [ ]:
pred = [ tolabel(p) for p in predictions]

In [ ]:
results = pd.DataFrame({"text":texts_subm, "label":pred})

In [ ]:
results[results['label']>0.03][results['label']<0.95]


In [ ]:
results.to_csv('results.csv', columns=[ 'label'])